In [1]:
import numpy as np
from tqdm import tqdm
import csv

In [102]:
#隐马尔可夫模型
class HMM_BIO:
    #param:
    # status 状态集合
    # observe 观测集合
    # (A,B,PI) 状态转移矩阵 发射矩阵 初始状态矩阵
    def __init__(self,A=None,B=None,PI=None):
        self.print=True
        
        
        self.N = 7     # 状态集合有多少元素(状态数BIO)
        self.M = 65535 # 观测集合有多少元素(汉字)
        
        #数字化 状态集合Q 观测集合V 
        self.status_dict={'B-PER': 0, # {'盒子1':0，'盒子2':1，'盒子3':2}
                        'I-PER': 1,
                        'B-LOC': 2,
                        'I-LOC': 3,
                        'B-ORG': 4,
                        'I-ORG': 5,
                        'O': 6}
        self.status=list(self.status_dict.keys())          
        #self.observe_dict={}# {'红':0，'白':1}
        self.Q=np.arange(0,self.N) #状态集合 [0,1,2]
        self.V=np.arange(0,self.M) #观测集合 [0,1]
        
        #初始化 (A,B,PI)
        self.A=A # 状态转移矩阵
        self.B=B # 发射矩阵
        self.PI=PI # 初始状态概率
        
        
        if self.print:
            print('状态集合status',self.status_dict)
            #print('观测集合observe',self.observe_dict)
            print('状态集合Q',self.Q)
            print('观测集合V',self.V)
            print()
        
    
    #计算前向概率
    #param
    # o 观测序列
    def calc_foward(self,o):
        print('calc_foward')
        #数字化 观测序列O
        O=np.array([self.observe_dict[x] for x in o])
        
        A=self.A
        B=self.B
        
        if self.print:
            print('观测序列O',O)
        
        #初始化alpha
        alpha=self.PI*(B.T[O[0]])
        if self.print:
            print('alpha0',alpha)
            
        #循环计算alpha
        for i in range(1,len(O)):
            alpha=alpha@A*(B.T[O[i]])
            if self.print:
                print(f'alpha{i}',alpha)
        ret=np.sum(alpha)
        print(f'前向概率:{ret}\n')
        return ret
    
    #计算后向概率
    def calc_backward(self,o):
        print('calc_backward')
        #数字化 观测序列O
        O=np.array([self.observe_dict[x] for x in o])
        
        A=self.A
        B=self.B
        
        if self.print:
            print('观测序列O',O)
        
        #初始化beta
        beta=np.ones(self.N)
        
        #循环计算beta
        for i in range(len(O)-1,0,-1):
            beta=A@(beta*(B.T[O[i]]))
            if self.print:
                print(f'beta{i}',beta)
        
        beta=self.PI*(beta*(B.T[O[0]]))
        if self.print:
            print('beta0',beta)
            
        ret=np.sum(beta)
        print(f'后向概率:{ret}\n')
        return ret
    
    #训练模型(A,B,PI)
    #data (2,N)
    def train_bio(self,data):
        self.calc_Param_bio(data)
    
    #利用数据data(状态序列+观测序列) 计算参数(A,B,PI) 直接使用统计方法 统计A,B,PI
    #param
    # data: (2,n)
    def calc_Param_bio(self,data):
        check(data)
        A=np.zeros((self.N,self.N)) #(N,N)
        B=np.zeros((self.N,self.M)) #(N,M)
        PI=np.zeros(self.N) #(N,)
        
        #d (2,n)状态和观测序列
        #row0 状态序列
        #row1 观测序列
        d=data
        n=d.shape[1]

        PI[d[0,0]]+=1
        B[d[0,0],d[1,0]]+=1

        for j in tqdm(range(1,n)):
            #当前为分隔符
            if d[0,j]==-1:
                continue
            if d[0,j-1]!=-1:
                #d[0,i] 当前状态
                #d[0,i-1] 上一次状态
                #[1,i] 当前观测

                A[d[0,j-1],d[0,j]]+=1
                B[d[0,j],d[1,j]]+=1
            #上一个为分隔符
            else:
                PI[d[0,j]]+=1
                B[d[0,j],d[1,j]]+=1
        
        self.A=A
        self.B=B
        self.PI=PI
        
        self.normalize_param()
        print('训练参数结果(A,B,PI)')
        print(f'A:{self.A}')
        print(f'B:{self.B}')
        print(f'PI:{self.PI}')
        
    #将(A,B,PI)归一化
    def normalize_param(self):
        self.A=self.A/np.sum(self.A,axis=1,keepdims=True)
        self.B=self.B/np.sum(self.B,axis=1,keepdims=True)
        self.PI=self.PI/np.sum(self.PI)

    
    
    #维特比算法(动态规划)
    #已知(A,B,PI) 和一观测序列o 求解其状态序列的概率最大解
    #param:
    # o 观测序列 [0,1,1,1,1,0] shape(n,)
    #return:
    # [1,2,0,0,0] 一段状态序列
    def viterbi_t(self,o):
        n=o.shape[0]
        dp=np.zeros((self.N,n)) #shape(状态个数,序列长度)
        A=self.A
        B=self.B
        PI=self.PI
        
        dp[:,0]=PI*B[:,o[0]]
        for i in range(1,n):
            dp[:,i]=np.max(dp[:,i-1]*A.T,axis=1)*B[:,o[i]]
        #check(dp)
        return np.argmax(dp,axis=0)
    
    #计算a,b阵列的准确率
    #param:
    # a 源标签
    # b 目的标签
    def calc_acc(self,label,predict):
        return np.mean(np.equal(label,predict))
    
    #在已有的参数(A,B,PI)下随机生成一组长度为n的 状态和观测序列
    #return:
    # np.ndarray[] shape=(2,n)
    # row0 状态序列
    # row1 观测序列
    def generate_Sequence(self,n):
        PI=self.PI
        A=self.A
        B=self.B
        
        ret=np.zeros((2,n),dtype=np.int32)
        
        #从状态集中以概率P随机选择一个状态
        ret[0,0]=np.random.choice(self.Q,p=PI)
        #从该状态生成一个观测值
        ret[1,0]=np.random.choice(self.V,p=B[ret[0,0]])
        
        
        for i in range(1,n):
            ret[0,i]=np.random.choice(self.Q,p=A[ret[0,i-1]])
            ret[1,i]=np.random.choice(self.V,p=B[ret[0,i]])
        return ret
    
    #生成 N组序列 序列中状态长度为n shape=(N,2,n)
    def generate_Data(self,N,n):
        ret=np.array([])
        for i in range(N):
            a=self.generate_Sequence(n)
            a=a.reshape(1,a.shape[0],a.shape[1])
            if i==0:
                ret=a
            else:
                ret=np.r_[ret,a]
        return ret
    
    
    #数字化观测序列o 将['red','white','red']转换为[0,1,0]
    def observe_transform(self,o):
        return np.array([self.observe_dict[x] for x in o])
    
    #预测一段话的状态序列
    #param
    # s 一段中文
    #return
    # 中文加状态的文字 林B-LOC_|徽I-LOC_|因_O_|
    def predict(self,s:str):
        ans=""
        s=list(s)
        o=np.array([ord(ch) for ch in s])
        print(o)
        statusList=self.viterbi_t(o)
        for i in range(len(s)):
            ans+=s[i]+str(self.status[statusList[i]])+'_|'
        #print(statusList)
        return ans
        

In [47]:
def train_BIO(hmm):
    file_path='corpus/BIO_train.txt'
    with open(file_path,mode='r',encoding='utf-8') as fr:
        lines=fr.readlines()
    for i in range(len(lines)):
        if len(lines[i])==1:
            continue
        else:
            ch,tag=lines.split('\t')
            
def data_pretreatment(hmm:HMM_BIO):
    file_path='corpus/BIO_train.txt'
    with open(file_path,mode='r',encoding='utf-8') as fr:
        lines=fr.readlines()
    with open('corpus/BIO_train_pretreatment.csv',mode='w',encoding='utf-8',newline='') as fw:
        writer=csv.writer(fw)
        for i in tqdm(range(len(lines))):
            #判断是否是换行\n
            if len(lines[i])==1:
                writer.writerow([-1,-1])
            else:
                ch,tag=lines[i].split()
                writer.writerow([hmm.status_dict[tag],ord(ch)])
                #new_row=np.array([[hmm.status_dict[tag],ord(ch)]])     

In [98]:
#Test predict
hmm_bio1=HMM_BIO()
hmm_bio1.predict('嘉然今天吃什么')


状态集合status {'B-PER': 0, 'I-PER': 1, 'B-LOC': 2, 'I-LOC': 3, 'B-ORG': 4, 'I-ORG': 5, 'O': 6}
状态集合Q [0 1 2 3 4 5 6]
观测集合V [    0     1     2 ... 65532 65533 65534]

[22025 28982 20170 22825 21507 20160 20040]


TypeError: 'NoneType' object is not subscriptable

In [103]:
hmm_bio=HMM_BIO()

状态集合status {'B-PER': 0, 'I-PER': 1, 'B-LOC': 2, 'I-LOC': 3, 'B-ORG': 4, 'I-ORG': 5, 'O': 6}
状态集合Q [0 1 2 3 4 5 6]
观测集合V [    0     1     2 ... 65532 65533 65534]



In [62]:
#将文本数据预处理 写进csv文件
data_pretreatment(hmm_bio)

100%|██████████| 2220537/2220537 [00:02<00:00, 1043495.94it/s]


In [63]:
#将csv文件中数据读出为np数据
data=np.genfromtxt('corpus/BIO_train_pretreatment.csv',delimiter=',',dtype=np.int32)
check(data)
data=data.T

[[    6 24403]
 [    6 24076]
 [    6 26395]
 ...
 [    6 19978]
 [    6 12290]
 [   -1    -1]]
数据类型 <class 'numpy.ndarray'>
数组元素数据类型： int32
数组元素总数： 4441074
数组形状： (2220537, 2)
数组的维度数目 2


## 运行下面

In [104]:
hmm_bio.train_bio(data)

[[    6     6     6 ...     6     6    -1]
 [24403 24076 26395 ... 19978 12290    -1]]
数据类型 <class 'numpy.ndarray'>
数组元素数据类型： int32
数组元素总数： 4441074
数组形状： (2, 2220537)
数组的维度数目 2


100%|██████████| 2220536/2220536 [00:05<00:00, 401999.77it/s]

训练参数结果(A,B,PI)
A:[[4.03134227e-03 9.45378151e-01 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 5.05905065e-02]
 [4.37553226e-03 5.13023816e-01 8.80979649e-05 0.00000000e+00
  4.99221801e-04 0.00000000e+00 4.82013332e-01]
 [0.00000000e+00 0.00000000e+00 4.34615911e-02 8.35820896e-01
  3.01246063e-04 0.00000000e+00 1.20416267e-01]
 [1.29565147e-03 0.00000000e+00 3.08324561e-02 3.82176694e-01
  1.27540692e-03 0.00000000e+00 5.84419791e-01]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  8.26486460e-04 9.95575867e-01 3.59764694e-03]
 [1.29236071e-03 0.00000000e+00 4.39163316e-03 0.00000000e+00
  3.47022784e-04 7.55121578e-01 2.38847406e-01]
 [7.62659866e-03 0.00000000e+00 1.60233671e-02 0.00000000e+00
  9.00129320e-03 0.00000000e+00 9.67348741e-01]]
B:[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
PI:[5.73255952e-02 5.92206562e-05 5.82928659e-02 0.000

In [124]:
s = '林徽因什么理由拒绝了徐志摩而选择梁思成为终身伴侣？' \
        '谢娜为李浩菲澄清网络谣言，之后她的两个行为给自己加分'
s1='到哈尔滨了'
print(hmm_bio.predict(s1))
print(hmm_bio.predict(s))

[21040 21704 23572 28392 20102]
到O_|哈B-LOC_|尔I-PER_|滨I-LOC_|了O_|
[26519 24509 22240 20160 20040 29702 30001 25298 32477 20102 24464 24535
 25705 32780 36873 25321 26753 24605 25104 20026 32456 36523 20276 20387
 65311 35874 23068 20026 26446 28009 33778 28548 28165 32593 32476 35875
 35328 65292 20043 21518 22905 30340 20004 20010 34892 20026 32473 33258
 24049 21152 20998]
林B-PER_|徽I-PER_|因O_|什O_|么O_|理O_|由O_|拒O_|绝O_|了O_|徐B-PER_|志I-PER_|摩I-PER_|而O_|选O_|择O_|梁B-PER_|思I-PER_|成O_|为O_|终O_|身O_|伴O_|侣O_|？O_|谢O_|娜I-PER_|为O_|李B-PER_|浩I-PER_|菲I-PER_|澄I-PER_|清I-PER_|网O_|络O_|谣O_|言O_|，O_|之O_|后O_|她O_|的O_|两O_|个O_|行O_|为O_|给O_|自O_|己O_|加O_|分O_|


In [6]:
#查看数组a1的属性
def check(a1):
    print(a1)  
    print("数据类型",type(a1))           #打印数组数据类型  
    print("数组元素数据类型：",a1.dtype) #打印数组元素数据类型  
    print("数组元素总数：",a1.size)      #打印数组尺寸，即数组元素总数  
    print("数组形状：",a1.shape)         #打印数组形状  
    print("数组的维度数目",a1.ndim)      #打印数组的维度数目

In [125]:
print(hmm_bio.A)

[[4.03134227e-03 9.45378151e-01 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 5.05905065e-02]
 [4.37553226e-03 5.13023816e-01 8.80979649e-05 0.00000000e+00
  4.99221801e-04 0.00000000e+00 4.82013332e-01]
 [0.00000000e+00 0.00000000e+00 4.34615911e-02 8.35820896e-01
  3.01246063e-04 0.00000000e+00 1.20416267e-01]
 [1.29565147e-03 0.00000000e+00 3.08324561e-02 3.82176694e-01
  1.27540692e-03 0.00000000e+00 5.84419791e-01]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  8.26486460e-04 9.95575867e-01 3.59764694e-03]
 [1.29236071e-03 0.00000000e+00 4.39163316e-03 0.00000000e+00
  3.47022784e-04 7.55121578e-01 2.38847406e-01]
 [7.62659866e-03 0.00000000e+00 1.60233671e-02 0.00000000e+00
  9.00129320e-03 0.00000000e+00 9.67348741e-01]]


In [126]:
print(hmm_bio.B)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [127]:
print(hmm_bio.PI)

[5.73255952e-02 5.92206562e-05 5.82928659e-02 0.00000000e+00
 6.97619330e-02 5.92206562e-05 8.14501165e-01]
